In [1]:
import requests

In [2]:
import geopandas as gpd
import pandas as pd

## Open State data

In [3]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
state_gdf = gpd.read_file(file_path)

## Get Columm Data

In [4]:
r = requests.get("https://api.census.gov/data/2023/acs/acs5/subject/variables.json")
columns_obj = r.json()

In [7]:
columns_to_replace = {
    key: val["label"]
    for key, val in columns_obj["variables"].items()
    if key.startswith("S0801")
}

In [18]:
mean_time_cols_dict = {
    col_name: col_dict
    for col_name, col_dict in columns_obj["variables"].items()
    if col_dict["label"].split("!!")[-1] == "Mean travel time to work (minutes)"
    and col_name.startswith("S0801")
}

In [19]:
mean_time_cols_dict

{'S0801_C02_046E': {'label': 'Estimate!!Male!!Workers 16 years and over who did not work from home!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)',
  'concept': 'Commuting Characteristics by Sex',
  'predicateType': 'float',
  'group': 'S0801',
  'limit': 0,
  'attributes': 'S0801_C02_046EA,S0801_C02_046M,S0801_C02_046MA'},
 'S0801_C01_046E': {'label': 'Estimate!!Total!!Workers 16 years and over who did not work from home!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)',
  'concept': 'Commuting Characteristics by Sex',
  'predicateType': 'float',
  'group': 'S0801',
  'limit': 0,
  'attributes': 'S0801_C01_046EA,S0801_C01_046M,S0801_C01_046MA'},
 'S0801_C03_046E': {'label': 'Estimate!!Female!!Workers 16 years and over who did not work from home!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)',
  'concept': 'Commuting Characteristics by Sex',
  'predicateType': 'float',
  'group': 'S0801',
  'limit': 0,
  'attributes': 'S0801_C03_046EA,S0801_C03_046M,S0801_C0

### Get columns to query and rename for later

In [20]:
columns = ["GEO_ID", "S0801_C01_046E", "S0801_C01_046M"]
columns_formatted = ",".join(columns)

In [21]:
response = requests.get(
    f"https://api.census.gov/data/2023/acs/acs5/subject?get={columns_formatted}&for=state:*"
)

In [24]:
response

<Response [200]>

In [22]:
data = response.json()
columns = data[0]
rows = data[1:]
commuting_df = pd.DataFrame(rows, columns=columns)

In [26]:
commuting_df = commuting_df[columns]
commuting_df[columns[1:]] = commuting_df[columns[1:]].astype(float)

In [28]:
columns_to_replace["GEO_ID"] = "AFFGEOID"
commuting_df = commuting_df.rename(columns=columns_to_replace)

In [29]:
commuting_df.columns

Index(['AFFGEOID',
       'Estimate!!Total!!Workers 16 years and over who did not work from home!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)',
       'S0801_C01_046M', 'state'],
      dtype='object')

## Merge Data

In [30]:
commuting_gdf = state_gdf.merge(commuting_df, on="AFFGEOID", how="inner")

In [31]:
commuting_gdf = commuting_gdf.to_crs(9311)
commuting_gdf.to_file("data/Average_Travel_Time_To_Work_Per_State.gpkg")